# Census "hard to count" analysis

By [Ben Welsh](https://palewi.re/who-is-ben-welsh/)

This data preparation routine was developed for the April 29, 2019, Los Angeles Times story ["A census undercount could cost California billions — and L.A. is famously hard to track"](https://www.latimes.com/local/lanow/la-me-la-county-census-hard-to-count-20190429-htmlstory.html).

It combines the California Department of Finance's "hard to count" estimates with the tract maps published by the U.S. Census Bureau. Together they were used to make a graphic to accompany the story.

## How we did it

Download the state's "hard to count" esimates.

In [1]:
!python download.py htc

Download the Census Bureau's tract maps.

In [2]:
!python download.py tracts

2019-08-16 18:18:15,046|tracts|DEBUG|ZIP file already exists at /home/jovyan/notebooks/la_times/census-hard-to-map-analysis/data/tracts/tl_2010_06_tract10.zip
2019-08-16 18:18:15,047|tracts|DEBUG|Unzipping /home/jovyan/notebooks/la_times/census-hard-to-map-analysis/data/tracts/tl_2010_06_tract10.zip to /home/jovyan/notebooks/la_times/census-hard-to-map-analysis/data/tracts


Import Python tools

In [3]:
import pandas as pd
import geopandas as gpd

Read in the hard-to-count data

In [4]:
# Added by Steve
pd.read_excel('./data/htc/tracts.xlsx').dtypes

GEOID           int64
CA HTC Index    int64
dtype: object

*Added by Steve*

tracts.xlsx maps geoid, unique identifies for census track data to HTC estimates from the California Census authority. `tracts.xlsx` thus represents third-party analysis. As the government has already determined a metric for these areas.

In [5]:
df = pd.read_excel("./data/htc/tracts.xlsx", dtype={"GEOID": str})

In [6]:
df.head()

,GEOID,CA HTC Index
0,06001400100,20
1,06001400200,16
2,06001400300,31
3,06001400400,35
4,06001400500,47


Clean it up.

In [7]:
df_trimmed = df.rename(columns={
    "GEOID": "geoid",
    "CA HTC Index": "htc_index"
})

In [8]:
df_trimmed.head()

,geoid,htc_index
0,06001400100,20
1,06001400200,16
2,06001400300,31
3,06001400400,35
4,06001400500,47


Read in the tract maps.

In [9]:
gdf = gpd.read_file("data/tracts/tl_2010_06_tract10.shp")

In [10]:
gdf.head()

,STATEFP10,COUNTYFP10,TRACTCE10,GEOID10,NAME10,NAMELSAD10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry
0,06,083,002103,06083002103,21.03,Census Tract 21.03,G5020,S,2838200,7603,+34.9306689,-120.4270588,"POLYGON ((-120.417938 34.938341, -120.417658 3..."
1,06,083,002402,06083002402,24.02,Census Tract 24.02,G5020,S,16288573,44468,+34.9287963,-120.4780833,"POLYGON ((-120.473893 34.920814, -120.474285 3..."
2,06,083,002102,06083002102,21.02,Census Tract 21.02,G5020,S,1352551,0,+34.9421111,-120.4267767,"POLYGON ((-120.417658 34.938345, -120.417938 3..."
3,06,083,002010,06083002010,20.10,Census Tract 20.10,G5020,S,2417990,0,+34.8714281,-120.4100285,"POLYGON ((-120.411468 34.879619, -120.411413 3..."
4,06,083,002009,06083002009,20.09,Census Tract 20.09,G5020,S,2603281,0,+34.8722878,-120.4277159,"POLYGON ((-120.423524 34.879283, -120.422856 3..."


Clean it up.

In [11]:
gdf_trimmed = gdf[[
    'GEOID10',
    'geometry'
]].rename(columns={
    "GEOID10": "geoid",
})

In [12]:
gdf_trimmed.head()

,geoid,geometry
0,06083002103,"POLYGON ((-120.417938 34.938341, -120.417658 3..."
1,06083002402,"POLYGON ((-120.473893 34.920814, -120.474285 3..."
2,06083002102,"POLYGON ((-120.417658 34.938345, -120.417938 3..."
3,06083002010,"POLYGON ((-120.411468 34.879619, -120.411413 3..."
4,06083002009,"POLYGON ((-120.423524 34.879283, -120.422856 3..."


Merge the data and the map

In [21]:
# Added by Steve, is this a true inner join (intersection of two sets)? Answer: yep
dfMinusGdf = set(df_trimmed.geoid.unique()).difference(set(gdf_trimmed.geoid.unique()))
GdfMinusDf = set(gdf_trimmed.geoid.unique()).difference(set(df_trimmed.geoid.unique()))
print('{}\n{}'.format(dfMinusGdf, GdfMinusDf))

{'06037137000'}
{'06037930401'}


In [13]:
merged_gdf = gdf_trimmed.merge(df_trimmed, on="geoid", how="inner")

Output the merged file for a graphic

In [11]:
merged_gdf.to_file("data/processed/tracts.shp")

How many of the hardest to count are here in LA County?

In [12]:
merged_gdf['county_fips'] = merged_gdf.geoid.str.slice(2, 5)

In [17]:
merged_gdf.county_fips.value_counts().head()

037    2345
073     628
059     583
065     453
085     372
Name: county_fips, dtype: int64

In [14]:
top_100 = merged_gdf.sort_values("htc_index", ascending=False).head(100)

In [18]:
top_100.head()

,geoid,geometry,htc_index,county_fips
3473,06077000100,"POLYGON ((-121.292051 37.95407, -121.293315 37...",136,077
7102,06037212305,"POLYGON ((-118.2998 34.057707, -118.29871 34.0...",128,037
6966,06037209300,"POLYGON ((-118.271663 34.053097, -118.2714 34....",127,037
7270,06037231710,"POLYGON ((-118.287222 34.010102, -118.28722 34...",123,037
3472,06077000300,"POLYGON ((-121.292051 37.95407, -121.291901 37...",123,077


In [19]:
top_100.county_fips.value_counts()

037    57
077     7
019     5
075     5
071     5
073     4
025     3
067     3
029     3
053     2
001     2
059     1
095     1
047     1
099     1
Name: county_fips, dtype: int64